In [26]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os


def extract_data(span):
    """강력한 태그로부터 텍스트를 추출합니다."""
    return span.strong.text.strip() if span and span.strong else ''

def scrape_data():
    url = "https://academy.prompie.com/schools/school-district/list/"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"Request failed: {e}")
        return

    soup = BeautifulSoup(response.text, 'html.parser')

    # 데이터 저장을 위한 리스트
    school_districts = []
    snu_admission_rates = []
    college_admission_rates = []
    special_high_school_admission_rates = []
    academy_counts = []

    # 상위 30개 학군 처리
    for item in soup.select('span.font-weight-semi-bold'):
        school_districts.append(item.text.strip())
        data_div = item.find_parent('h5').find_next_sibling('div')
        data_items = data_div.find_all('span', class_='font-weight-normal')
        
        # 각 데이터 추출
        snu_admission_rates.append(data_items[0].strong.text.strip())
        college_admission_rates.append(data_items[1].strong.text.strip())
        special_high_school_admission_rates.append(data_items[2].strong.text.strip())
        academy_counts.append(data_items[3].strong.text.strip())

        
    # 31위부터 100위까지 처리
    table_body = soup.find('tbody', class_='main-table-body')
    rows = table_body.find_all('tr')  

    for row in rows[0:70]:  # Adjust the slice as necessary
        rank_cell = row.find('h6', class_='font-weight-normal')
        rank = int(rank_cell.text) if rank_cell else None  
        cells = row.find_all('td')

        if len(cells) == 6:  
            school_districts.append(cells[1].get_text(strip=True))
            snu_admission_rates.append(cells[2].get_text(strip=True))
            college_admission_rates.append(cells[3].get_text(strip=True))
            special_high_school_admission_rates.append(cells[4].get_text(strip=True))
            academy_counts.append(cells[5].get_text(strip=True))



    # 데이터프레임 생성 및 파일 저장
    df = pd.DataFrame({
        '학군지': school_districts,
        '서울대입학률': snu_admission_rates,
        '4년제입학률': college_admission_rates,
        '특목/자사고진학률': special_high_school_admission_rates,
        '학원수': academy_counts
    })

    excel_file = 'school_data1.xlsx'
    
    try:
        df.to_excel(excel_file, index=False)
    except PermissionError:
        print(f"The file {excel_file} is open. Please close it before running the script.")
        return

    print(df.head())


In [23]:
scrape_data()

          학군지 서울대입학률  4년제입학률 특목/자사고진학률     학원수
0  서울 강남구 대치동  3.75%  96.66%    27.13%  1,442개
1  서울 서초구 반포동  4.20%  95.07%    36.02%    456개
2  부산 금정구 구서동  1.75%  96.16%    29.66%    266개
3   서울 양천구 목동  2.41%  90.68%    24.65%  1,022개
4  부산 해운대구 우동  0.99%  94.35%    18.75%    306개
File is closed and ready to be overwritten.


In [43]:
df = pd.read_excel("school_data.xlsx")
df.head()

,학군지,서울대입학률,4년제입학률,특목/자사고진학률,학원수
0,서울 강남구 대치동,3.75%,96.66%,27.13%,"1,442개"
1,서울 서초구 반포동,4.20%,95.07%,36.02%,456개
2,부산 금정구 구서동,1.75%,96.16%,29.66%,266개
3,서울 양천구 목동,2.41%,90.68%,24.65%,"1,022개"
4,부산 해운대구 우동,0.99%,94.35%,18.75%,306개


In [44]:
df['서울대입학률'] = df['서울대입학률'].str.rstrip('%').astype(float)
df['4년제입학률'] = df['4년제입학률'].str.rstrip('%').astype(float)
df['특목/자사고진학률'] = df['특목/자사고진학률'].str.rstrip('%').astype(float)
df['학원수'] = df['학원수'].str.replace(',', '').str.rstrip('개').astype(int)

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   학군지        100 non-null    object 
 1   서울대입학률     100 non-null    float64
 2   4년제입학률     100 non-null    float64
 3   특목/자사고진학률  100 non-null    float64
 4   학원수        100 non-null    int32  
dtypes: float64(3), int32(1), object(1)
memory usage: 3.6+ KB


In [46]:
df.head()

,학군지,서울대입학률,4년제입학률,특목/자사고진학률,학원수
0,서울 강남구 대치동,3.75,96.66,27.13,1442
1,서울 서초구 반포동,4.20,95.07,36.02,456
2,부산 금정구 구서동,1.75,96.16,29.66,266
3,서울 양천구 목동,2.41,90.68,24.65,1022
4,부산 해운대구 우동,0.99,94.35,18.75,306


In [47]:
# 띄어쓰기가 세 개 이상인 경우 처리하는 함수
def extract_location(row):
    parts = row['학군지'].split(' ')
    if len(parts) >= 3:
        return parts[0], parts[1], ' '.join(parts[2:])
    elif len(parts) == 2:
        return parts[0], parts[1], ''
    else:
        return '', parts[0], ''

# apply 함수를 사용하여 시, 구, 동 열 생성
df['시'], df['구'], df['동'] = zip(*df.apply(extract_location, axis=1))

In [49]:
df.head(20)

,학군지,서울대입학률,4년제입학률,특목/자사고진학률,학원수,시,구,동
0,서울 강남구 대치동,3.75,96.66,27.13,1442,서울,강남구,대치동
1,서울 서초구 반포동,4.20,95.07,36.02,456,서울,서초구,반포동
2,부산 금정구 구서동,1.75,96.16,29.66,266,부산,금정구,구서동
3,서울 양천구 목동,2.41,90.68,24.65,1022,서울,양천구,목동
4,부산 해운대구 우동,0.99,94.35,18.75,306,부산,해운대구,우동
5,경기 성남시 분당구 정자동,2.26,86.03,21.96,455,경기,성남시,분당구 정자동
6,서울 강남구 개포동,3.21,87.93,30.65,183,서울,강남구,개포동
7,대전 서구 둔산동,1.62,87.41,17.46,438,대전,서구,둔산동
8,서울 강남구 역삼동,2.32,86.34,13.11,405,서울,강남구,역삼동
9,서울 강남구 삼성동,2.94,91.96,19.77,138,서울,강남구,삼성동


In [52]:
# "시", "구", "동"을 분리하는 함수
def split_address(row):
    parts = row.split() # 띄어쓰기 기준으로 분할
    city, district, neighborhood = '', '', ''

    for part in parts:
        if '시' in part or '도' in part: # "시" 또는 "도"가 포함된 경우
            city += ' ' + part
            city = city.strip()
        elif '구' in part: # "구"가 포함된 경우
            district = part
        elif '동' in part: # "동"이 포함된 경우
            neighborhood = part
    
    return pd.Series([city, district, neighborhood])

# 새로운 열 추가
df[['시', '구', '동']] = df['학군지'].apply(split_address)

df

,학군지,서울대입학률,4년제입학률,특목/자사고진학률,학원수,시,구,동
0,서울 강남구 대치동,3.75,96.66,27.13,1442,,강남구,대치동
1,서울 서초구 반포동,4.20,95.07,36.02,456,,서초구,반포동
2,부산 금정구 구서동,1.75,96.16,29.66,266,,구서동,
3,서울 양천구 목동,2.41,90.68,24.65,1022,,양천구,목동
4,부산 해운대구 우동,0.99,94.35,18.75,306,,해운대구,우동
...,...,...,...,...,...,...,...,...
95,경기 의정부시 녹양동,1.74,87.89,6.19,52,의정부시,,녹양동
96,경기 고양시 일산서구 대화동,0.57,73.56,8.81,146,고양시,일산서구,대화동
97,경기 구리시 교문동,0.86,74.86,10.64,85,구리시,,교문동
98,전북 전주시 완산구 효자동2가,0.61,87.71,3.70,163,전주시,완산구,효자동2가
